In [1]:
from dask.distributed import Client, LocalCluster
import bokeh
import geopandas as gpd
import scipy
import libpysal
import momepy
import json
from dask import delayed
import glob
import dask
import dask.dataframe as dd
import momepy
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import json
from tqdm import tqdm
from shapely.geometry import Point
import networkx as nx
import pickle

/tmp/ipykernel_4906/3967175050.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
daskCluster = LocalCluster(threads_per_worker=2,
                n_workers=8, memory_limit='100GB')

client = Client(daskCluster)

client

2023-04-18 22:20:10,291 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-18 22:20:10,294 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-18 22:20:10,297 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-18 22:20:10,299 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-18 22:20:10,302 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-18 22:20:10,304 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-18 22:20:10,307 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-18 22:20:10,

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 250.47 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42779,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 250.47 GiB
Comm: tcp://127.0.0.1:38939,Total threads: 2
Dashboard: http://127.0.0.1:45535/status,Memory: 31.31 GiB
Nanny: tcp://127.0.0.1:38833,


In [3]:
# with open('./out/singapore/what_cells_are_in_what_cluster_dict.json', 'r') as f:
#     includes_dict = json.load(f)
    
tessellation = gpd.read_parquet("./out/singapore/tessellation.pq")

buildings = gpd.read_parquet("./out/singapore/buildings.pq")

streets = gpd.read_parquet("./out/singapore/streets.pq").explode()

/tmp/ipykernel_4906/394842713.py:8: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  streets = gpd.read_parquet("./out/singapore/streets.pq").explode()


In [4]:
streets['nID'] = momepy.unique_id(streets)
# buildings['nID'] = momepy.get_network_id(buildings, streets,
#                                                'nID').values

# buildings = buildings.drop_duplicates("uID").drop(columns="index_right")
# tessellation = tessellation.merge(buildings[['uID', 'nID']], on='uID', how='left')

In [5]:
### Distance Weight
dist200 = libpysal.weights.DistanceBand.from_dataframe(buildings, 200, ids='uID')

/home/virgilxw/miniconda3/lib/python3.10/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 520 disconnected components.
 There are 195 islands with ids: 99, 113, 921, 1567, 1722, 1724, 2523, 2536, 2559, 2562, 2566, 2612, 2619, 2630, 3467, 3476, 3482, 3510, 3513, 3514, 3515, 3516, 3524, 4055, 4177, 4178, 4188, 4189, 4238, 4670, 4819, 4820, 4947, 5488, 5502, 5611, 5639, 5651, 5672, 5676, 6193, 6322, 6436, 6576, 7127, 7131, 7439, 7440, 7449, 7540, 8379, 8380, 8741, 8960, 9128, 9215, 9216, 9220, 9221, 9222, 9283, 9292, 10240, 12427, 12978, 13708, 14446, 15110, 15172, 15173, 15176, 16024, 16029, 16677, 16694, 16695, 17122, 17419, 17960, 18648, 18666, 18692, 21202, 22013, 22669, 22670, 24504, 24514, 24515, 25723, 26327, 26343, 29028, 29033, 29037, 29038, 29891, 31014, 32389, 32390, 32566, 33077, 33157, 33935, 34204, 34473, 34920, 35640, 36329, 41504, 42003, 42592, 44147, 44304, 44305, 44744, 45136, 45137, 45151, 45584, 45587, 4

In [6]:
# @dask.delayed
# def queen_contiguity_load(n1, includes_dict):
#     chunk = gpd.read_parquet(f"./out/singapore/expanded_cells_in_chunk_{int(n1)}.pq")
    
#     # weights
    # queen_1 = libpysal.weights.w_subset(momepy.sw_high(k=1, gdf=chunk, ids='uID'), includes_dict[str(n1)]).to_adjlist()
    # queen_3 = libpysal.weights.w_subset(momepy.sw_high(k=3, gdf=chunk, ids='uID'), includes_dict[str(n1)]).to_adjlist()
    
#     return (queen_1, queen_3)

In [7]:
# file_pattern = "./out/singapore/expanded_cells_in_chunk_*.pq"
# file_list = glob.glob(file_pattern)
# num_files = len(file_list)

# weight_out = [process(n1, includes_dict) for n1 in tqdm(range(num_files))]

# weight_out = [queen_contiguity_load(n1, includes_dict) for n1 in range(num_files)]

# weight_out = dask.compute(weight_out)


In [8]:
# queen_1 = libpysal.weights.W.from_adjlist(pd.concat(weight_out[0][i][0] for i in range(num_files)))

# queen_3 = libpysal.weights.W.from_adjlist(pd.concat(weight_out[0][i][1] for i in range(num_files)))

In [9]:
tessellation_bridged = gpd.read_parquet("./out/singapore/tessellation_bridges.pq")

In [10]:
queen_1 = libpysal.weights.contiguity.Queen.from_dataframe(tessellation_bridged, ids="uID", silence_warnings=True)
queen_3 = momepy.sw_high(k=3, weights=queen_1)

In [11]:
# # Find the uIDs in tessellation that are not in buildings
# tessellation_uids = set(tessellation['uID'])
# buildings_uids = set(buildings['uID'])
# new_uids = tessellation_uids - buildings_uids

# # Create a new GeoDataFrame with the new uIDs and empty geometry
# new_rows = pd.DataFrame({'uID': list(new_uids)})
# new_rows['geometry'] = None
# new_gdf = gpd.GeoDataFrame(new_rows, crs=buildings.crs)

# # Append the new GeoDataFrame to buildings
# new_buildings = pd.concat([buildings, new_gdf], ignore_index=True)

In [12]:
### Building Area
buildings['area'] = momepy.Area(buildings).series

## Building Shapes

### Circular Compactness
buildings['circular_com'] =  momepy.CircularCompactness(buildings).series

### Elongation
blg_elongation = momepy.Elongation(buildings)
buildings['elongation'] = blg_elongation.series

## Squareness
blg_squareness = momepy.Squareness(buildings)
buildings['squareness'] = blg_squareness.series

buildings['eri'] = momepy.EquivalentRectangularIndex(buildings).series

buildings['area'] = momepy.Area(buildings).series


buildings['orientation'] = momepy.Orientation(buildings).series

  0%|          | 0/76533 [00:00<?, ?it/s]

/home/virgilxw/miniconda3/lib/python3.10/site-packages/momepy/shape.py:962: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type == "Polygon":
/home/virgilxw/miniconda3/lib/python3.10/site-packages/momepy/shape.py:956: RuntimeWarning: invalid value encountered in arccos
  angle = np.degrees(np.arccos(cosine_angle))
/home/virgilxw/miniconda3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/virgilxw/miniconda3/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


  0%|          | 0/76533 [00:00<?, ?it/s]

/home/virgilxw/miniconda3/lib/python3.10/site-packages/momepy/distribution.py:74: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type in ["Polygon", "MultiPolygon", "LinearRing"]:


In [13]:
tessellation["convexity"] = momepy.Convexity(tessellation).series
tessellation["area"] = momepy.Area(tessellation).series

tessellation["neighbours"] = momepy.Neighbors(tessellation, queen_3,'uID').series
tessellation["neighbours400"] = momepy.Neighbors(tessellation, dist200, 'uID').series
tessellation["covered_area"] = momepy.CoveredArea(tessellation, queen_1, "uID", verbose=False).series

areas = momepy.Area(tessellation).series
tessellation["mean_area"] = momepy.AverageCharacter(
tessellation, values=areas, spatial_weights=queen_3, unique_id='uID').mean

tessellation["mean_area_iq"] = momepy.AverageCharacter(
    tessellation, areas, queen_3, 'uID', rng=(25, 75)).mean

tessellation["area_rng"] = momepy.Range(tessellation, values='area',
                                        spatial_weights=queen_3, unique_id='uID').series
tessellation["area_iqr"] = momepy.Range(tessellation, values='area',
                        spatial_weights=queen_3, unique_id='uID',
                        rng=(25, 75)).series
tessellation["rea_theil"] = momepy.Theil(tessellation, values='area',
                        spatial_weights=queen_3,
                        unique_id='uID').series

tessellation["shannon"] = momepy.Shannon(tessellation, values='area',spatial_weights=queen_3, unique_id='uID').series


  0%|          | 0/76498 [00:00<?, ?it/s]

  0%|          | 0/76498 [00:00<?, ?it/s]

  0%|          | 0/76498 [00:00<?, ?it/s]

  0%|          | 0/76498 [00:00<?, ?it/s]

  0%|          | 0/76498 [00:00<?, ?it/s]

  0%|          | 0/76498 [00:00<?, ?it/s]

  0%|          | 0/76498 [00:00<?, ?it/s]

  0%|          | 0/76498 [00:00<?, ?it/s]

In [14]:
edg_lin = momepy.Linearity(streets)
streets['linearity'] = edg_lin.series

profile = momepy.StreetProfile(streets, buildings)
streets["width"] = profile.w
streets["width_deviation"] = profile.wd
streets["openness"] = profile.o

/home/virgilxw/miniconda3/lib/python3.10/site-packages/momepy/shape.py:1286: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type == "LineString"


In [15]:
# primal = momepy.gdf_to_nx(streets, approach='primal')

In [16]:
%%time
# momepy.node_degree(primal)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.77 µs


In [17]:
%%time
# primal = momepy.closeness_centrality(primal, radius=400, name='closeness400', distance='mm_len', weight='mm_len')

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.2 µs


In [18]:
%%time
# primal = momepy.meshedness(primal, radius=400, distance="mm_len")

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.68 µs


In [19]:
# %%time
# primal = momepy.straightness_centrality(primal)

In [20]:
# %%time
# primal = momepy.closeness_centrality(primal, name='closeness_global', weight='mm_len')

In [21]:
# %%time
# primal =  momepy.betweenness_centrality(primal, name='betweenness_metric_n', mode='nodes', weight='mm_len', k=100)

In [22]:
# %%time
# primal = momepy.straightness_centrality(primal)
# # momepy.mean_nodes(primal, 'straightness')
# momepy.mean_nodes(primal, 'closeness400')
# # momepy.mean_nodes(primal, 'closeness_global')
# # momepy.mean_nodes(primal, 'betweenness_metric_n')

# nodes, streets = momepy.nx_to_gdf(primal)

In [23]:
# dual = momepy.gdf_to_nx(streets, approach='dual')

# dual = momepy.closeness_centrality(dual, radius=5, name='angcloseness400', weight='angle')

# # dual = momepy.closeness_centrality(dual, name='closeness_global_ang', weight='angle')

# # dual = momepy.betweenness_centrality(dual, name='angbetweenness', mode='nodes', weight='angle')

# edges_d = momepy.nx_to_gdf(dual)


In [24]:
# Find the uIDs in tessellation that are not in buildings
tessellation_uids = set(tessellation['uID'])
buildings_uids = set(buildings['uID'])
new_uids = tessellation_uids - buildings_uids

# Create a new GeoDataFrame with the new uIDs and empty geometry
new_rows = pd.DataFrame({'uID': list(new_uids)})
new_rows['geometry'] = None
new_gdf = gpd.GeoDataFrame(new_rows, crs=buildings.crs)

# Append the new GeoDataFrame to buildings
buildings = pd.concat([buildings, new_gdf], ignore_index=True)

In [25]:
buildings.head()

,geometry,uID,area,circular_com,elongation,squareness,eri,orientation
0,"POLYGON ((4098.105 23706.883, 4109.145 23706.8...",0.0,135.936426,0.632538,0.896277,0.028693,1.000012,0.004613
1,"POLYGON ((4169.087 23688.953, 4169.269 23731.3...",1.0,940.600377,0.523132,0.523543,0.012337,1.000018,0.245322
2,"POLYGON ((4129.786 23795.994, 4196.953 23795.9...",2.0,3326.535481,0.608176,0.737375,0.000006,1.000000,0.004602
3,"POLYGON ((3940.171 23898.114, 3939.741 23952.6...",3.0,1412.582668,0.493066,0.474393,0.012235,1.000018,0.451083
4,"POLYGON ((3908.968 23952.996, 3934.866 23953.1...",4.0,1412.739688,0.493120,0.474631,0.009556,0.999984,0.451164


In [26]:
circular_compactness = momepy.CircularCompactness(buildings)
buildings['weighted_circom'] = momepy.WeightedCharacter(
buildings, circular_compactness.series, queen_3, 'uID', momepy.Area(buildings).series).series

buildings['neighbour_dist'] = momepy.NeighborDistance(buildings, queen_1, 'uID').series

buildings['interbuilding_distance'] = momepy.MeanInterbuildingDistance(buildings, queen_1, 'uID', 3, verbose=False).series

interblg_distance = momepy.MeanInterbuildingDistance(
buildings, queen_1, 'uID', spatial_weights_higher=queen_3)
buildings['mean_ib_dist'] = interblg_distance.series

buildings['mean_ib_dist'] = momepy.MeanInterbuildingDistance(
    buildings, queen_1, 'uID', order=3).series




  0%|          | 0/76533 [00:00<?, ?it/s]

  0%|          | 0/76533 [00:00<?, ?it/s]

Computing mean interbuilding distances...


  0%|          | 0/76533 [00:00<?, ?it/s]

Computing mean interbuilding distances...


  0%|          | 0/76533 [00:00<?, ?it/s]

In [27]:
cols = list(streets.columns)
cols.remove('nID')
cols.remove('geometry')

In [28]:
buildings['weighted_circom'] = momepy.WeightedCharacter(
buildings, circular_compactness.series, queen_3, 'uID', momepy.Area(buildings).series).series

  0%|          | 0/76533 [00:00<?, ?it/s]

In [29]:
buildings = buildings.sjoin_nearest(streets.drop(cols,axis=1).reset_index(drop=True), max_distance=1000, how="left")

/home/virgilxw/miniconda3/lib/python3.10/site-packages/geopandas/geodataframe.py:2173: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:3414

  return geopandas.sjoin_nearest(


In [30]:
buildings = buildings.dropna(subset=['geometry'])
tessellation = tessellation.dropna(subset=['geometry'])

In [31]:
buildings['orientation'] = momepy.Orientation(buildings).series
tessellation['orientation'] = momepy.Orientation(tessellation).series

  0%|          | 0/81694 [00:00<?, ?it/s]

/home/virgilxw/miniconda3/lib/python3.10/site-packages/momepy/distribution.py:74: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type in ["Polygon", "MultiPolygon", "LinearRing"]:


  0%|          | 0/76498 [00:00<?, ?it/s]

In [32]:
str_align = momepy.StreetAlignment(buildings, streets,
                                   'orientation', 'nID',
                                   'nID')
buildings['str_align'] = str_align.series

/home/virgilxw/miniconda3/lib/python3.10/site-packages/momepy/distribution.py:84: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  elif geom.type in ["LineString", "MultiLineString"]:


In [33]:
blg_cell_align = momepy.CellAlignment(buildings, tessellation,
                                      'orientation', 'orientation',
                                      'uID', 'uID')
buildings['cell_align'] = blg_cell_align.series

In [34]:
tessellation['car'] = momepy.AreaRatio(tessellation, buildings, 'area', 'area', 'uID').series

In [35]:
nodes = gpd.read_parquet("./out/singapore/vertices_frame.pq")
edges = gpd.read_parquet("./out/singapore/edges_frame.pq")

In [36]:
buildings = buildings.drop(["uID", "nID", "index_right"], axis=1)
nodes = nodes.drop(["vertID"], axis=1)
edges = edges.drop("edgeID", axis=1)
buildings = buildings.sjoin_nearest(nodes.reset_index(drop=True), max_distance=1000, how="left").drop(["index_right"], axis=1)
buildings = buildings.sjoin_nearest(edges.reset_index(drop=True), max_distance=1000, how="left")

In [37]:
tessellation = tessellation.merge(buildings, how='left', left_on = "uID", right_index=True )
tessellation = tessellation.drop(['eID', "geometry_y", "index_right"], axis=1)
tessellation = tessellation.rename(columns={"geometry_x": "geometry"})
tessellation = gpd.GeoDataFrame(tessellation.dropna(subset=['geometry']))

In [ ]:
with open('./out/singapore/weights.pickle', 'wb') as f:
    pickle.dump(queen_1, f)
    pickle.dump(queen_3, f)
    pickle.dump(dist200, f)
    f.close()

In [ ]:
# tessellation.to_parquet("./out/singapore/tessellation_with_data.pq")